# Fairify Artifact
Artifact of the paper "Fairify: Fairness Verification of Neural Networks" from ICSE 2023.

To access ChameleonCloud resources, you may need the account to log in to ChameleonCloud. You also need to have a project to allocate resources (e.g., node).

In [38]:
from chi import context

context.version = "1.0"

context.choose_site(default="CHI@TACC")
context.choose_project()

### Check available hardware

In [39]:
from chi import hardware

node_type = "compute_cascadelake_r"
available_nodes = hardware.get_nodes(node_type=node_type, filter_reserved=True)
if available_nodes:
    print(f"There currently are {len(available_nodes)} {node_type} nodes ready to use")
else:
    print(f"All {node_type} nodes are in use! You could use next_free_timeslot to see how long you need to wait, or use the calendar.")

There currently are 38 compute_cascadelake_r nodes ready to use


### Reserve node

In [40]:
from chi import lease
from datetime import timedelta
import os

my_lease = lease.Lease(f"{os.getenv('USER')}-power-management", duration=timedelta(hours=8))
my_lease.add_node_reservation(nodes=[available_nodes[0]]) # or you could use node_type=node_type
my_lease.add_fip_reservation(1) # include a floating ip
my_lease.submit(idempotent=True)

Waiting for lease to start...


Lease radhofanazizi_gmail_com-power-management has reached status active


### Create a server on the node

In [41]:
from chi import server

my_server = server.Server(
    f"{os.getenv('USER')}-power-management",
    reservation_id=my_lease.node_reservations[0]["id"],
    image_name="CC-Ubuntu22.04", # or use image_name
)
my_server.submit(idempotent=True)

Waiting for server radhofanazizi_gmail_com-power-management's status to become ACTIVE. This typically takes 10 minutes for baremetal, but can take up to 20 minutes.


Server has moved to status ACTIVE


Attribute,radhofanazizi_gmail_com-power-management
Id,48cfbc15-555b-459d-a1ea-9555bf3741bf
Status,ACTIVE
Image Name,CC-Ubuntu22.04
Flavor Name,baremetal
Addresses,sharednet1: IP: 10.52.0.204 (v4) Type: fixed MAC: bc:97:e1:78:f1:a0
Network Name,sharednet1
Created At,2025-07-03T07:46:33Z
Keypair,trovi-28d289c
Reservation Id,0f352bac-e686-4e9a-b218-6ae9323e07b8
Host Id,b281b13a05d4a4d342f673906de4005142c2819a049809e34ac97306


### Configure networking on the node

In [42]:
fip = my_lease.get_reserved_floating_ips()[0]
my_server.associate_floating_ip(fip)
my_server.check_connectivity(host=fip)

Checking connectivity to 129.114.109.67 port 22.


Connection successful


# Setup nvidia drivers and cuda

Download drivers

In [ ]:
my_server.execute("""
sudo apt-get --purge remove -y '*cublas*' 'cuda*' 'nvidia-*' && \
sudo apt-get autoremove -y && \
sudo apt-get autoclean && \
sudo rm -rf /usr/local/cuda* && \
sudo apt update && \
sudo apt install -y nvidia-driver-470 && \
wget https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2004/x86_64/cuda-keyring_1.0-1_all.deb && \
sudo dpkg -i cuda-keyring_1.0-1_all.deb && \
sudo apt update && \
sudo apt install -y cuda-toolkit-11-2
""")

Restart nvidia services

In [ ]:
my_server.execute("sudo modprobe -r nouveau")
my_server.execute("sudo rmmod nouveau 2>/dev/null || true")
my_server.execute("sudo update-initramfs -u")
my_server.execute("sudo modprobe nvidia")

Verifiy services

In [ ]:
my_server.execute("nvidia-smi")
my_server.execute("""
if [ -L /usr/local/bin/nvcc ]; then sudo rm /usr/local/bin/nvcc; fi && \
sudo ln -s /usr/local/cuda-11.2/bin/nvcc /usr/local/bin/nvcc
""")
my_server.execute("nvcc --version")

## Run Fairify

Now, we can finally run Fairify. First we need to clone the github repo first and then run the reprduce.sh script which contain the instructions from README.md
packaged into a bash file

In [45]:
my_server.execute("rm -rf Fairify && git clone https://github.com/radhofan/Fairify.git")

Cloning into 'Fairify'...


<Result cmd='rm -rf Fairify && git clone https://github.com/radhofan/Fairify.git' exited=0>

In [47]:
my_server.execute("chmod +x Fairify/reproduce-experiment.sh")
my_server.execute("bash Fairify/reproduce-experiment.sh")

ERROR: File or directory already exists: '/home/cc/miniconda'
If you want to update an existing installation, use the -u option.


Channels:
 - conda-forge
 - defaults
Platform: linux-64
Solving environment: done

# All requested packages already installed.

Running `shell init`, which:
 - modifies RC file: "/home/cc/.bashrc"
 - generates config for root prefix: "/home/cc/miniconda"
 - sets mamba executable to: "/home/cc/miniconda/bin/mamba"
The following has been added in your "/home/cc/.bashrc" file

# >>> mamba initialize >>>
# !! Contents within this block are managed by 'mamba shell init' !!
export MAMBA_EXE='/home/cc/miniconda/bin/mamba';
export MAMBA_ROOT_PREFIX='/home/cc/miniconda';
__mamba_setup="$("$MAMBA_EXE" shell hook --shell bash --root-prefix "$MAMBA_ROOT_PREFIX" 2> /dev/null)"
if [ $? -eq 0 ]; then
    eval "$__mamba_setup"
else
    alias mamba="$MAMBA_EXE"  # Fallback on help from mamba activate
fi
unset __mamba_setup
# <<< mamba initialize <<<



warning  libmamba 'repo.anaconda.com', a commercial channel hosted by Anaconda.com, is used.
    
warning  libmamba Please make sure you understand Anaconda Terms of Services.
    
warning  libmamba See: https://legal.anaconda.com/policies/en/




Transaction

  Prefix: /home/cc/miniconda/envs/fairify

  Updating specs:

   - python=3.9


  Package               Version  Build           Channel         Size
───────────────────────────────────────────────────────────────────────
  Install:
───────────────────────────────────────────────────────────────────────

  + _libgcc_mutex           0.1  main            pkgs/main     Cached
  + _openmp_mutex           5.1  1_gnu           pkgs/main     Cached
  + bzip2                 1.0.8  h5eee18b_6      pkgs/main     Cached
  + ca-certificates   2025.2.25  h06a4308_0      pkgs/main     Cached
  + expat                 2.7.1  h6a678d5_0      pkgs/main     Cached
  + ld_impl_linux-64       2.40  h12ee557_0      pkgs/main     Cached
  + libffi                3.4.4  h6a678d5_1      pkgs/main     Cached
  + libgcc-ng            11.2.0  h1234567_1      pkgs/main     Cached
  + libgomp              11.2.0  h1234567_1      pkgs/main     Cached
  + libstdcxx-ng         11.2.0  h1234567_1      

Reading package lists...
Building dependency tree...
Reading state information...
csvtool is already the newest version (2.4-1build3).
0 upgraded, 0 newly installed, 0 to remove and 0 not upgraded.


...

Hybrid Prediction:   0%|          | 0/6784 [00:00<?, ?it/s]

        ...  
6779    False
6780    False
6781    False
6782     True
6783    False
Length: 6784, dtype: bool
True: 991 | False: 5793
prot_attr
0       0.0
1       1.0
2       1.0
3       1.0
4       1.0
       ... 
6779    1.0
6780    0.0
6781    0.0
6782    0.0
6783    1.0
Length: 6784, dtype: float64
INTERVAL BASED PRUNING
SINGULAR VERIFICATION
Pruning done!
Verifying ...
sat
V time:  20.384
['100', '0', '0', '3', '6', '2', '5', '1', '1', '1', '9', '100', '0']
['100', '0', '0', '3', '6', '2', '5', '1', '0', '1', '9', '100', '0']
[0.01599002] [-0.0424459]
[0.0159905] [-0.04244566]
pred1:  0.5039974195395943
pred2:  0.4893901183830316
class_1:  True
class_2:  False
pred1_orig:  0.5039975387412641
pred2_orig:  0.48939017796083784
class_1_orig:  True
class_2_orig:  False
******************
7 column
0       0.0
1       1.0
2       1.0
3       1.0
4       1.0
       ... 
6779    1.0
6780    0.0
6781    0.0
6782    0.0
6783    1.0
Name: 8, Length: 6784, dtype: float64
y_true
0       0
1   

2025-07-03 10:08:43.792393: I tensorflow/compiler/mlir/mlir_graph_optimization_pass.cc:176] None of the MLIR Optimization Passes are enabled (registered 2)
2025-07-03 10:08:43.811318: I tensorflow/core/platform/profile_utils/cpu_utils.cc:114] CPU Frequency: 2400000000 Hz
Hybrid Prediction: 100%|██████████| 6784/6784 [08:03<00:00, 14.04it/s]


AC-2 (Original) Accuracy: 0.8470
AC-15 (Fairer) Accuracy: 0.8470
Hybrid Approach Accuracy: 0.8470
=== CNT RESULTS ===
Original CNT:  0.9064
Fairer CNT:    0.9064
Hybrid CNT:    0.9064
Discrimination rate on original model (AC-2): 0.0110
Discrimination rate on fairer model   (AC-15): 0.0000
Discrimination rate on hybrid model: 0.0000

Debug Counters:
fallback_to_original: 4386
sat_unfair_ac-15_used: 0
unsat_fair_ac-2_used: 0
unknown_ac-2_used: 2598


<Result cmd='bash Fairify/reproduce-experiment.sh' exited=0>


## Run dependencies and first experiment

Results for first experiment

In [ ]:
from chi import storage

bucket_name = "bare_metal_experiment_pattern_data"
b = storage.ObjectBucket(bucket_name)

print(f"Listing objects in bucket '{bucket_name}'...")
for obj in b.list_objects():
    print(f"Downloading {obj.name}")
    obj.download(obj.name) 

In [ ]:
my_server.execute("csvtool readable Fairify/src/AC/res/counterexample-adult-new.csv | less -S")

In [ ]:
my_server.execute("csvtool readable Fairify/src/AC/res/AC-3.csv | less -S")

In [ ]:
my_server.execute("csvtool readable Fairify/src/AC/res/synthetic-adult-predicted-AC-metrics.csv | less -S")